In [1]:
import os
os.getcwd()

'c:\\Desktop\\AgenticSQL\\code'

In [2]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [3]:
## download chinook database

## it is a small database with tables related to music, albums, track etc. so we can use it with sqlite.

import requests

url = "https://github.com/tharani001/MyDatasets/raw/refs/heads/main/db_samples/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    with open("../database/Chinook.db", "wb") as file:
        file.write(response.content)
    print("Chinook.db downloaded successfully.")
else:
    print(f"Failed to download Chinook.db. Status code: {response.status_code}")

Chinook.db downloaded successfully.


### Load Database with Langchain SQLDatabase

In [6]:
# sqlite is small database that provides similar functionality to mysql and postgresql.

from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../database/Chinook.db")
# print(db.table_names())
print(db.dialect)
print(db.get_table_info()) 

sqlite

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCE

In [8]:
print(db.run("SELECT * FROM Album LIMIT 5;"))

[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]


### LLM initialization

In [9]:
# Qwen2.5 is twice the size of LLAMA3.2 model but it performs better on code related tasks.

from langchain_ollama import ChatOllama

model = "qwen2.5:7b"

base_url = "http://localhost:11434"

llm  = ChatOllama(model=model, base_url=base_url)

llm

ChatOllama(model='qwen2.5:7b', base_url='http://localhost:11434')

### Building Graph State 

In [10]:
# Graph is workflow that controls the how an agent thinks and acts by managing the flow of states between the nodes.

# state is just a dictionary that contains the messages.

# each node takes a state and returns a new state. it gets updated with the new state.

from typing import TypedDict

class State(TypedDict):
    question : str
    query : str
    result : str
    answer : str

### SQL Query prompt from Langchain hub

In [11]:
from langchain import hub

query_prompt_template = hub.pull('langchain-ai/sql-query-system-prompt')

query_prompt_template

ChatPromptTemplate(input_variables=['dialect', 'input', 'table_info', 'top_k'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'sql-query-system-prompt', 'lc_hub_commit_hash': '360a0e9d0f0f5da0ee9810a2a0ea3c9dc3de31b3ae9d50272420c33e48e6e323'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'table_info', 'top_k'], input_types={}, partial_variables={}, template='Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema

In [15]:
query_prompt_template.input_variables, query_prompt_template.messages[0].pretty_print()

================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}


(['dialect', 'input', 'table_info', 'top_k'], None)

### Output Parser

In [16]:
from typing import Annotated

class QueryOutput(TypedDict):
    """Generated SQL query"""

    query: Annotated[str, ..., "Syntactically correct and valid SQL query"]

QueryOutput({"query": "SELECT * FROM album LIMIT 2"})
QueryOutput.__annotations__

{'query': typing.Annotated[str, Ellipsis, 'Syntactically correct and valid SQL query']}

In [20]:
llm.with_structured_output(QueryOutput)

RunnableBinding(bound=ChatOllama(model='qwen2.5:7b', base_url='http://localhost:11434'), kwargs={'format': {'title': 'QueryOutput', 'description': 'Generated SQL query', 'type': 'object', 'properties': {'query': {'description': 'Syntactically correct and valid SQL query', 'type': 'string'}}, 'required': ['query']}, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema'}, 'schema': {'title': 'QueryOutput', 'description': 'Generated SQL query', 'type': 'object', 'properties': {'query': {'description': 'Syntactically correct and valid SQL query', 'type': 'string'}}, 'required': ['query']}}}, config={}, config_factories=[])
| JsonOutputParser()

### Write node: It turns question to a SQL Query

In [25]:
def write_query(state: State) -> State:
    """Generate MySQL query to fetch information"""

    prompt = query_prompt_template.invoke({
        "dialect": db.dialect,
        "top_k": 5,
        "table_info": db.get_table_info(),
        "input": state["question"]
    })
     
    output_parse_llm = llm.with_structured_output(QueryOutput)

    result = output_parse_llm.invoke(prompt)

    return {"query" : result["query"]}



In [26]:
write_query({"question": " How many albums are there in the database?"})

{'query': 'select count(*) from album'}

### Execute Query Node:  It generates the result

In [27]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

def execute_query(state:State):
    """Execute SQL query and return the result"""
    query = state["query"]
    execute_query_tool = QuerySQLDataBaseTool(db=db)

    return {'result': execute_query_tool.invoke({"query": query})}

In [28]:
execute_query({"query": "SELECT * FROM album LIMIT 2"})

C:\Users\thara\AppData\Local\Temp\ipykernel_24432\4185605201.py:6: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query_tool = QuerySQLDataBaseTool(db=db)


{'result': "[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2)]"}